(correspsearch-api)=

# Data Gathering und Data Wrangling mit Daten der CorrespSearch-API

:::{index} correspSearch
:name: correspsearch_
:::

Wenn wir mit Daten arbeiten, ist es eine immer wiederkehrende Aufgabe, diese Daten zunächst zu sammeln und sie für ein Format bzw. für ein Datenmodell aufzubereiten, wie es zur Weiterverarbeitung erforderlich ist. Im Folgenden führen wir diese Schritte exemplarisch durch. Dazu nutzen wir für das {term}`Data Gathering` die {term}`API` des Projekts [correspSearch](https://correspsearch.net/). CorrespSearch bietet zahlreiche {term}`Metadaten` zu einer Fülle an Briefkorrespondenzen. Wir werden uns auf die Briefkorrespondenz einer Person beschränken und werden die Metadaten zu den [Briefen von Alexander von Humboldt](https://correspsearch.net/de/suche.html?s=http://d-nb.info/gnd/118554700) mit Hilfe der API abrufen. Allerdings werden wir für diesen Zweck hier im Juypter Book nicht alle ca. 7.000 bei correspSearch verzeichneten Briefe nutzen, sondern lediglich diejenigen, die im Volltext der [edition humboldt digital](https://edition-humboldt.de/) verfügbar sind - diese Volltexte werden wir im nächsten Kapitel dann auswerten. Wie das Vorgehen ist, um an die Metadaten dieser Briefe heranzukommen, werden wir im Folgenden exemplarisch durchgehen.

## Install und Import

Für das hier gewählte Vorgehen können wir ein kleines Python-Paket sehr gut nutzen, dass uns bei der Arbeit mit {term}`JSON`-Objekten hilft. Zunächst installieren wir das Paket [`flatten-json`](https://pypi.org/project/flatten-json/) mit Hilfe von {term}`PIP`. Um den Befehl in einer Zelle des Jupyter Notebooks auszuführen, muss dem Befehl pip einfach ein '!' vorangestellt werden. Danach können wir dieses Paket mit den anderen erforderlichen Paketen importieren.

Wer nochmal einen Refresher zu JSON benötigt, erfährt dazu alles grundlegende im Jupyter Book 'Python Basics' im Kapitel [Dateien verarbeiten](https://digital-history-berlin.github.io/Python-fuer-Historiker-innen/ch04-dateien-verarbeiten/03-json.html).

:::{index} single: Bibliothek ; flatten-json
:name: flatten_json_
:::

:::{index} single: Bibliothek ; os
:name: os_
:::

In [ ]:
!pip install flatten-json

In [ ]:
import os
import json
import time
import requests

import pandas as pd

from flatten_json import flatten

## API von correspSearch

Die Dokumentation zur [correspSearch API v2](https://correspsearch.net/de/api.html) bietet alle wichtigen Informationen, um die Metadaten der in corresSearch vorhandenen Briefeditionen automatisiert abzufragen. Die Abfrage erfolgt mittels URL-Parameter: Mit Hilfe der {term}`Parameter` kann nach Personen/Institutionen, Orten, nach Zeiträumen, Berufen der Personen, nach den Editionen sowie nach Verfügbarkeit gefiltert werden. In der Dokumentation finden sich neben den Erläuterung zu diesen Parametern auch die nötigen Informationen über die unterschiedlichen Rückgabeformate. Für das im Folgenden vorgestellte Vorgehen haben wir uns für das Format {term}`TEI-JSON` entschieden. Weitere Erläuterungen zum Vorgehen bei einer API-Anfragen können im Kapitel "[Einführung in Weg APIs](intro-api)" dieses Jupyter Books nachgelesen werden.

Bevor wir uns die Zusammensetzung der API-Anfrage zuwenden, schauen wir uns die Daten, die wir zurückerhalten, etwas genauer an. Im `teiHeader` sind allgemeine Metadaten zum Projekt enthalten. Für uns ist das `notesStmt` interessant, denn hier können wir herausfinden, wie viele Treffer unsere Anfrage ergeben hat. In `sourceDesc` finden sich die Informationen zu den Editionen / Publikationen aus denen die Metadaten zu den Briefen entnommen sind. Wir interessieren uns aber vor allem für die Infos in `correspDesc`, den Beschreibungen der angefragten Briefe, die in unter `profileDesc` stehen. Grundsätzlich sind eine Erkundung der Datenmodellierung sowie gute Kenntnisse der Daten von Vorteil und stets sehr hilfreich.

```json
{
    "teiHeader": {
        "fileDesc": {
            "titleStmt": {
                "title": "correspSearch API 2.0 (BETA)",
                "editor": [
                    {
                        "#text": "correspsearch@bbaw.de",
                        "email": "no-email-provided@correspsearch.net"
                    }
                ]
            },
            "publicationStmt": {
                "publisher": [
                    {
                        "ref": {
                            "target": "https://www.bbaw.de/",
                            "#text": "Berlin-Brandenburg Academy of Sciences and Humanities"
                        }
                    }
                ],
                "availability": {
                    "licence": [
                        {
                            "target": "https://creativecommons.org/licenses/by/4.0/",
                            "#text": "CC-BY 4.0"
                        }
                    ]
                },
                "idno": "https://correspsearch.net/api/v2.0/tei-json.xql?s=http://d-nb.info/gnd/118554700&amp;e=d238c5bd-a978-475e-8784-58ce7ec82010",
                "date": {
                    "when": "2024-04-18T09:49:17.468+02:00"
                }
            },
            "notesStmt": {
                "note": "1-10 of 523 hits",
                "relatedItem": {
                    "type": "next",
                    "target": "https://correspsearch.net/api/v2.0/tei-json.xql?s=http://d-nb.info/gnd/118554700&amp;e=d238c5bd-a978-475e-8784-58ce7ec82010&amp;x=2"
                }
            },
            "sourceDesc": {
                "bibl": [
                    {
                        "xml:id": "d238c5bd-a978-475e-8784-58ce7ec82010",
                        "type": "online",
                        "#text": "edition humboldt digital, hg. v. Ottmar Ette. Berlin-Brandenburgische Akademie der Wissenschaften, Berlin 2017–2023.",
                        "ref": {
                            "target": "https://edition-humboldt.de",
                            "#text": "https://edition-humboldt.de"
                        }
                    }
                ]
            }
        },
```

```json
        "profileDesc": {
            "correspDesc": [
                {
                    "ref": "https://edition-humboldt.de/H0002656",
                    "source": "#d238c5bd-a978-475e-8784-58ce7ec82010",
                    "correspAction": [
                        {
                            "type": "sent",
                            "persName": [
                                {
                                    "ref": "http://d-nb.info/gnd/118554700",
                                    "#text": "Alexander von Humboldt"
                                }
                            ],
                            "placeName": [
                                {
                                    "ref": "http://sws.geonames.org/2911298",
                                    "#text": "Hamburg"
                                }
                            ],
                            "date": [
                                {
                                    "from": "1791-01-28",
                                    "to": "1791-02-20"
                                }
                            ]
                        },
                        {
                            "type": "received",
                            "persName": [
                                {
                                    "ref": "http://d-nb.info/gnd/118805193",
                                    "#text": "Samuel Thomas von Soemmerring"
                                }
                            ]
                        }
                    ]
                },
```

## Konstruktion der API-Anfrage

Nachdem wir uns gerade die Metadaten zum ersten Brief angesehen haben, betrachten wir nun die Konstruktion der API-Anfrage genauer. 

* Zunächst setzen wir die Variable `api_base_url` auf das Rückgabeformat `TEI-JSON`.

* Dann nutzen wir für die Variable `person` die GND-ID für Alexander von Humboldt.

* Da wir direkt bei der Anfrage einen ersten Filter einsetzen möchten, mit dem wir nur die Briefe aus der [edition humboldt digital](https://edition-humboldt.de/) zurückerhalten, wählen wir für den Parameter `e=` die ID für diese Edition aus und weisen diese der Variable `edition` zu.

Wo finden wir aber die ID, die für die Edition bzw. den Publikationsort der Briefe, die uns interessieren, steht? Bei einer allgemeine API-Anfrage nach Briefen von Alexander von Humboldt müssen wir uns die JSON-Datei genauer anschauen. Hier finden wir bei den entsprechenden Briefen und `sourceDesc` die Informationen zu dem entsprechenden Datengeber. Unter `sourceDesc`, `bibl` und dann `xml:id` finden wir einen langen String, der die ID für die entsprechende Edition darstellt. Diesen String können wir bei einer neuen Anfrage an die API nutzen, um nach Treffern, die nur von dieser Edition stammen, zu filtern.

In [ ]:
# TEI-JSON
api_base_url = 'https://correspsearch.net/api/v2.0/tei-json.xql?'

# GND-ID für Alexander von Humboldt
person = 'http://d-nb.info/gnd/118554700'

# Parameter für Edition / Publikation
edition = 'd238c5bd-a978-475e-8784-58ce7ec82010'

Schließlich setzen wir die einzelnen Bausteine zur API-Anfrage zusammen. Angaben zur Filterung werden jeweils mit dem `&` angefügt. Mit `print` können wir den Anfrage-String ausgeben.

In [ ]:
api_query = f'{api_base_url}s={person}&e={edition}'
print(api_query)

## Vorbereiten der API-Anfrage

Als nächsten nutzen wir das Python-Modul `requests`, um die Anfrage einmalig zu stellen. Wir können dann nachsehen, wieviele Treffer die Anfrage ergeben hat, um für die Konstruktion der eigentlichen Anfrage eine wichtige Information zu erhalten. Denn: Pro Anfrage werden jeweils die Informationen für 100 Treffer zurückgegeben, wir müssen also eine Schleife nutzen -- und hierfür benötigen wir die Anzahl, wie oft diese Schleife durchlaufen werden soll. Dazu greifen wir über `['teiHeader']['fileDesc']['notesStmt']['note']` auf den String zu, der die Angaben zu den *hits* enthält. Den String splitten wir auf, nutzen das vorletzte Elemente der durch den `split`-Befehl entstehenden Liste, wandeln dieses in ein integer um und teilen diese Zahl mit der *floor division* durch 100 auf die abgerundete Ganzzahl, um dann noch 1 hinzu zu addieren.

Diesen so berechnten Wert nutzen wir wiederum für den URL-Parameter `x=`, der für die Paginierung steht. Auf diese Weise erhalten wir die Treffer in Blöcken zu jeweils 100 records.

In [ ]:
response = requests.get(api_query)

In [ ]:
from pprint import pprint

In [ ]:
response.json()['teiHeader']['fileDesc']['notesStmt']['note']

In [ ]:
record_pages = int(response.json()['teiHeader']['fileDesc']['notesStmt']['note'].split()[-2]) // 10 + 1
print(record_pages)

Mit der nächsten Code-Zelle legen wir einen `data`-Ordner an, um dort die JSON-Dateien zu speichern.

In [ ]:
path = 'data'

try:
    os.mkdir(path)
except:
    print('Ordner existiert bereits.')

## Durchführung der API-Anfrage

Nun können wir die API-Anfrage starten. Dazu durchlaufen wir die Schleife entsprechend der Anzahl der Treffer in hunderter Blöcken. Nach jedem Durchlauf wird der Paginierungs-Parameter `x=` um eins erhöht. Wir geben dann die Angabe über den Fortgang des Durchlaufs mit dem `print`-Befehl aus. Danach konstruieren wir einen Dateinamen; dazu nutzen wir wiederum die Laufvariable der Schleife. Die erhaltenen Daten speichern wir im JSON-Format ab. Zuletzt warten wir 5 Sekunden, um die API mit den Anfragen nicht zu stark zu belasten.

In [ ]:
for page_nr in range(1, record_pages + 1):

    response = requests.get(f'{api_query}&x={page_nr}')

    print(response.json()['teiHeader']['fileDesc']['notesStmt']['note'])

    with open(f'data/{str(page_nr).zfill(4)}_AvH_edi_hum_dig.json', 'w', encoding='utf-8') as f:
        json.dump(response.json(), f, ensure_ascii=False, indent=4)

    time.sleep(5)

Schauen wir uns den Inhalt des `data`-Ordners kurz an, ob alles passt.

In [ ]:
files = os.listdir('data')
print(len(files))
print(sorted(files))

## Von JSON zum Pandas Dataframe

Das nächste Ziel ist es, die Daten aus den JSON-Dateien in einen pandas Dataframe zu bringen. Die in der Variable `files` enthaltenen und sortierten Dateien durchlaufen wir mit einer for-Schleife: Wir öffnen die Datei, lesen die JSON ein und nutzen nun die Methode `flatten` aus dem Paket `flatten-json`, um mit Hilfe einer List Comprehension die verschachtelte Stuktur der Dictionaries 'einzuebnen'. Mit `extend()` fügen wir die jeweiligen Listen einer neu erstellten Liste hinzu. Diese Liste, die alle JSON-Objekte enthält, können wir nun in einen Dataframe überführen. Wir nutzen `flatten` allerdings lediglich für die Dictionaries, die in `correspDesc` enthalten sind. Daher setzen wir die Indexierung über `['teiHeader']['profileDesc']['correspDesc']` in den JSON-Objekten auf diesen für uns relevanten Abschnitt der Datei.

In [ ]:
all_json_obj_flattened = []

for file in sorted(files):

    if file.endswith('.json'):

        with open(f'data/{file}', 'r', encoding='utf8') as f:

            json_obj = json.load(f)

            dic_flattened = [ flatten(dic) for dic in json_obj['teiHeader']['profileDesc']['correspDesc'] ]

            all_json_obj_flattened.extend(dic_flattened)

In [ ]:
df = pd.DataFrame(all_json_obj_flattened)
print(df.shape)
df.head()

## Aufbereiten des Dataframes

Wir erhalten nun einen Dataframe mit 523 Zeilen und 28 Spalten. Mit `df.isna().sum()` können wir anzeigen, wie viele der Werte leer geblieben, also {term}`NANs` (not a number) sind. In den nächsten Schritten wollen wir diesen Dataframe weiter aufbereiten, denn wir wollen für die Zwecke dieses JupyterBooks einen etwas kleineren Datensatz nutzen. Um den Umfang zu reduzieren, werden wir nur die Datensätze der Briefe nutzen, deren Datierung genau angegeben ist. Diese Info finden wir in der Spalte `correspAction_0_date_0_when`, in der auch noch 227 Werte fehlen. 

In [ ]:
print(df.shape)
df.isna().sum()

Im Folgenden werden wir mit `dropna()` verschiedene Spalten und Zeilen löschen, in denen keine Werte enthalten sind. Wir nutzen das Argument `subset=`, um Spalten gezielt auszuwählen und wir verwenden auch das Argument `inplace=True`, um die Änderungen direkt an dem Dataframe zu vollziehen.

:::{index} single: pandas ; dropna()
:name: dropna_
:::

In [ ]:
df.dropna(subset=['correspAction_0_date_0_when'], inplace=True)
print(df.shape)

In der folgenden Zelle erstellen wir einen Dataframe, der die Informationen zu Ortsangaben zwischenspeichert. In der Zelle darauf können wir nun alle Spalten, die leere Werte enthalten, löschen. Wir erhalten nun einen Dataframe mit 296 Zeilen und nur noch 9 Spalten.

In [ ]:
df_place = df.loc[:, ['correspAction_0_placeName_0_ref', 'correspAction_0_placeName_0_#text']]
df_place.shape

In [ ]:
df.dropna(axis=1, inplace=True)
print(df.shape)

Ferner löschen wir noch Spalten, die die Info `sent` bzw. `receive` enthalten. Durch eine spätere Umbenennung der Spalten können wir diese Infos über Sender und Empfänger jedoch in einer anderen Spalten mit unterbringen. Nun haben wir nur noch 7 Spalten und alle 296 Zeilen enthalten einen Wert in den einzelnen Zellen.

In [ ]:
df.drop(['correspAction_0_type', 'correspAction_1_type'], axis=1, inplace=True)
print(df.shape)
print(df.isna().sum())

Im nächsten Schritt fügen wir die zwischengespeicherten Infos zu den Ortsangaben dem Dataframe mit `join()` wieder hinzu: Das Ergebnis ist ein Dataframe mit 296 Zeillen und 9 Spalten.

:::{index} single: pandas ; join()
:name: join_
:::

In [ ]:
df = df.join(df_place)
print(df.shape)
df.head()

In den letzten Schritten benennen wir die Spaltennamen um und reseten den Index der Zeilen, sodass wir eine saubere Durchnummerierung der Zeilen von 0 bis 295 erhalten. Zum Schluss speichern wir den Dataframe als csv-Datei ab.

:::{index} single: pandas ; reset_index()
:name: reset_index_
:::

In [ ]:
# rename columns
df.columns = ['reference', 'edition_id', 'sender_id', 'sender', 'receiver_id', 'receiver', 'date', 'place_id', 'place' ]

# reset index
df.reset_index(drop=True, inplace=True)

df.head()

Zum Schluss speichern wir den Dataframe als CSV-Datei ab.

:::{index} single: pandas ; to_csv()
:name: to_csv_
:::

In [ ]:
# save
df.to_csv('data/AvH-letters-with-date.csv', index=False)